<a href="https://colab.research.google.com/github/nyp-sit/iti121-2025s2/blob/main/L4/LIME_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install lime

In [7]:
from lime import lime_image
from skimage.segmentation import mark_boundaries
from torchvision import models, transforms
from PIL import Image
import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 1. Load pretrained model ---
model = models.resnet18(pretrained=True).to(device).eval()


Using device: cuda


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Preprocess image

Before passing the image to model, we need to preprocess image (transform, resize, etc) to what the model expected during its training.  For example, resnet expects the images to have mean of (0.485, 0.456, 0.406) and std deviation of (0.229, 0.224, 0.225), for each channel.

In [8]:
# --- 2. Preprocessing ---
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


We need to define a classifier function that LIME can use to get the predicted probabilities

In [9]:
def batch_predict(images):
    """Convert numpy arrays to tensor batch and predict"""
    batch = torch.stack([preprocess(Image.fromarray(img.astype('uint8')))
                         for img in images], dim=0)

    with torch.no_grad():
        logits = model(batch)
        probs = torch.nn.functional.softmax(logits, dim=1)

    return probs.cpu().numpy()


In [12]:
# --- 3. Load example image ---
img = Image.open("cockatoo.jpeg")

# --- 4. Initialize LIME explainer ---
explainer = lime_image.LimeImageExplainer()

# --- 5. Explain a prediction ---
explanation = explainer.explain_instance(
    np.array(img),
    classifier_fn=batch_predict,
    top_labels=1,  # LIME will only explain the top predicted label (the class with highest probability).
    hide_color=0, # When LIME “hides” a superpixel, it replaces its pixels with this value (color). In this case, it is black
    num_samples=1000 # Number of perturbed samples (versions of the image) to generate.
)



  0%|          | 0/1000 [00:00<?, ?it/s]

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

### Let's visualiza the image explanation.

In [10]:
# --- 6. Visualize result ---
from matplotlib import pyplot as plt
temp, mask = explanation.get_image_and_mask(
    label=explanation.top_labels[0],  # which class to explain
    positive_only=True, # Show only features (superpixels) that increase the probability of that class
    hide_rest=False, # If True, hide non-important regions (fill them with gray/black); if False, keep the full image visible
    num_features=5, # Number of most influential superpixels to highlight
    min_weight=0.0 # Minimum importance threshold
)

plt.imshow(mark_boundaries(temp / 255.0, mask))
plt.title("LIME Explanation")
plt.axis("off")
plt.show()

NameError: name 'explanation' is not defined